In [ ]:
import os

os.environ["ANTHROPIC_API_KEY"] = ""

In [2]:
import sys

sys.path.append("../../app/")
sys.path.append("../../")

# Metrics

In [3]:
import numpy as np
from sklearn.metrics import mean_absolute_error


def accuracy_at_threshold(gold_scores, pred_scores, threshold):
    gold_scores = np.array(gold_scores)
    pred_scores = np.array(pred_scores)
    absolute_errors = np.abs(gold_scores - pred_scores)
    within_threshold_count = np.sum(absolute_errors <= threshold)
    return within_threshold_count / len(gold_scores)


# Evaluate

In [4]:
import pandas as pd

train_df = pd.read_csv("../../data/train.csv")
val_df = pd.read_csv("../../data/test.csv")

## Kaggle dataset

In [ ]:
from langchain_anthropic import ChatAnthropic
from langgraph.graph import END, START, StateGraph

import app.workflow.node as node
from app.agents.evaluation import EvaluationAgent
from app.agents.jd_extractor import JDExtractor
from app.agents.resume_extractor import ResumeExtractor
from app.workflow.state import State

model = ChatAnthropic(model="claude-3-7-sonnet-20250219", temperature=0)
RESUME_EXTRACTOR = ResumeExtractor(llm=model)
EVALUATION_AGENT = EvaluationAgent(llm=model)
JD_EXTRACTOR = JDExtractor(llm=model)

CONFIGS = [
    {"obj": node.ValidateInputNode, "configs": {"name": "VALIDATION_INPUT"}},
    {
        "obj": node.ParseContentNode,
        "configs": {
            "name": "CONTENT_EXTRACTION",
            "jd_extractor": JD_EXTRACTOR,
            "resume_extractor": RESUME_EXTRACTOR,
        },
    },
    {
        "obj": node.EvaluatePairMatchingNode,
        "configs": {"name": "EVALUATION", "evaluation_agent": EVALUATION_AGENT},
    },
]

NODES: list[node.BaseNode] = [value["obj"](**value["configs"]) for value in CONFIGS]


def build_graph():
    graph = StateGraph(state_schema=State)
    for user_node in NODES:
        graph.add_node(user_node.name, user_node)

    graph.add_edge(START, "VALIDATION_INPUT")
    graph.add_edge("VALIDATION_INPUT", "CONTENT_EXTRACTION")
    graph.add_edge("CONTENT_EXTRACTION", "EVALUATION")
    graph.add_edge("EVALUATION", END)
    return graph


In [6]:
graph = build_graph()
app = graph.compile(checkpointer=None)

In [7]:
import time

from tqdm import tqdm

gold_scores = []
pred_scores = []

print(f"\n--- Starting Evaluation on {len(val_df)} examples ---")
batch_size = 30
for i in tqdm(
    range(0, len(val_df), batch_size),
    desc="Evaluating LangGraph in Batches",
    total=len(val_df) // batch_size,
):
    batch_df = val_df.iloc[i : i + batch_size]
    batch_gold_scores = batch_df["match_score"].tolist()

    # Run the LangGraph app
    state = State(
        inputs=[
            {
                "job_description": example["job_description"],
                "resume": example["resume"],
            }
            for _, example in batch_df.iterrows()
        ]
    )
    try:
        result = app.invoke(state)
        batch_pred_scores = [
            result["results"][j].get("score") for j in range(len(result["results"]))
        ]
    except Exception as e:
        print(f"An error occurred during graph execution: {e}")
        batch_pred_scores = [0.0] * len(batch_df)  # Assign a penalty score

    gold_scores.extend(batch_gold_scores)
    pred_scores.extend(batch_pred_scores)

    # To avoid rate limiting
    time.sleep(1)

print("--- Evaluation Complete ---")

# --- Calculate Metrics ---

# 1. Mean Absolute Error (MAE)
final_mae = mean_absolute_error(gold_scores, pred_scores)

# 2. Accuracy@threshold
acc_at_1 = accuracy_at_threshold(gold_scores, pred_scores, threshold=1)
acc_at_2 = accuracy_at_threshold(gold_scores, pred_scores, threshold=2)
acc_at_3 = accuracy_at_threshold(gold_scores, pred_scores, threshold=3)


# --- Print Results ---

print("\n--- LangGraph Evaluation Results ---")
print(f"📊 Mean Absolute Error (MAE): {final_mae:.4f}")
print("---")
print(f"🎯 Accuracy@1 (error <= 1.0): {acc_at_1:.2%}")
print(f"🎯 Accuracy@2 (error <= 2.0): {acc_at_2:.2%}")
print(f"🎯 Accuracy@3 (error <= 3.0): {acc_at_3:.2%}")
print("--------------------------------------")


--- Starting Evaluation on 3000 examples ---


Evaluating LangGraph in Batches:   0%|          | 0/100 [00:00<?, ?it/s]

[VALIDATION_INPUT] Validating input...
Pass id 0
Pass id 1
Pass id 2
Pass id 3
Pass id 4
Pass id 5
Pass id 6
Pass id 7
Pass id 8
Pass id 9
Pass id 10
Pass id 11
Pass id 12
Pass id 13
Pass id 14
Pass id 15
Pass id 16
Pass id 17
Pass id 18
Pass id 19
Pass id 20
Pass id 21
Pass id 22
Pass id 23
Pass id 24
Pass id 25
Pass id 26
Pass id 27
Pass id 28
Pass id 29
An error occurred during graph execution: 0


Evaluating LangGraph in Batches:   1%|          | 1/100 [00:33<55:03, 33.37s/it]

[VALIDATION_INPUT] Validating input...
Pass id 0
Pass id 1
Pass id 2
Pass id 3
Pass id 4
Pass id 5
Pass id 6
Pass id 7
Pass id 8
Pass id 9
Pass id 10
Pass id 11
Pass id 12
Pass id 13
Pass id 14
Pass id 15
Pass id 16
Pass id 17
Pass id 18
Pass id 19
Pass id 20
Pass id 21
Pass id 22
Pass id 23
Pass id 24
Pass id 25
Pass id 26
Pass id 27
Pass id 28
Pass id 29
An error occurred during graph execution: 0


...

[VALIDATION_INPUT] Validating input...
Pass id 0
Pass id 1
Pass id 2
Pass id 3
Pass id 4
Pass id 5
Pass id 6
Pass id 7
Pass id 8
Pass id 9
Pass id 10
Pass id 11
Pass id 12
Pass id 13
Pass id 14
Pass id 15
Pass id 16
Pass id 17
Pass id 18
Pass id 19
Pass id 20
Pass id 21
Pass id 22
Pass id 23
Pass id 24
Pass id 25
Pass id 26
Pass id 27
Pass id 28
Pass id 29
An error occurred during graph execution: 0


Evaluating LangGraph in Batches:  99%|█████████▉| 99/100 [54:28<00:34, 34.18s/it]

[VALIDATION_INPUT] Validating input...
Pass id 0
Pass id 1
Pass id 2
Pass id 3
Pass id 4
Pass id 5
Pass id 6
Pass id 7
Pass id 8
Pass id 9
Pass id 10
Pass id 11
Pass id 12
Pass id 13
Pass id 14
Pass id 15
Pass id 16
Pass id 17
Pass id 18
Pass id 19
Pass id 20
Pass id 21
Pass id 22
Pass id 23
Pass id 24
Pass id 25
Pass id 26
Pass id 27
Pass id 28
Pass id 29
An error occurred during graph execution: 0


Evaluating LangGraph in Batches: 100%|██████████| 100/100 [55:05<00:00, 33.06s/it]

--- Evaluation Complete ---

--- LangGraph Evaluation Results ---
📊 Mean Absolute Error (MAE): 6.9960
---
🎯 Accuracy@1 (error <= 1.0): 0.00%
🎯 Accuracy@2 (error <= 2.0): 3.93%
🎯 Accuracy@3 (error <= 3.0): 3.93%
--------------------------------------


## Collected dataset

In [8]:
import json
import time

from tqdm import tqdm

collected_folder = "/Users/nhannguyen/ngtuthanhan@gmail.com - Google Drive/My Drive/HCMUS/LLM/Data_Collector_30_8"
matching_json = os.path.join(collected_folder, "Matching_Result.json")

with open(matching_json, "r") as f:
    matching_data = json.load(f)

pred_scores = []
gold_scores = []

batch_size = 5
for i in tqdm(
    range(0, len(matching_data), batch_size),
    desc="Evaluating LangGraph in Batches",
    total=len(matching_data) // batch_size,
):
    batch_examples = matching_data[i : i + batch_size]
    batch_gold_scores = [example["Score"] for example in batch_examples]
    batch_jd = []
    batch_resume = []
    for example in batch_examples:
        jd_path = example["JD"]
        resume_path = example["CV"]
        gold_score = example["Score"]
        with open(os.path.join(collected_folder, "JD", jd_path) + ".txt", "r") as f:
            jd = f.read()
        with open(os.path.join(collected_folder, "CV", resume_path) + ".txt", "r") as f:
            resume = f.read()
        batch_jd.append(jd)
        batch_resume.append(resume)

    state = State(
        inputs=[
            {
                "job_description": jd,
                "resume": resume,
            }
            for jd, resume in zip(batch_jd, batch_resume)
        ]
    )
    try:
        result = app.invoke(state)
        pred_score = result["results"][0].get("score")
    except Exception as e:
        print(f"An error occurred during graph execution: {e}")
        pred_score = 0.0  # Assign a penalty score

    gold_scores.append(gold_score)
    pred_scores.append(pred_score)

    # To avoid rate limiting
    time.sleep(1)


print("--- Evaluation Complete ---")

# --- Calculate Metrics ---

# 1. Mean Absolute Error (MAE)
final_mae = mean_absolute_error(gold_scores, pred_scores)

# 2. Accuracy@threshold
acc_at_1 = accuracy_at_threshold(gold_scores, pred_scores, threshold=1)
acc_at_2 = accuracy_at_threshold(gold_scores, pred_scores, threshold=2)
acc_at_3 = accuracy_at_threshold(gold_scores, pred_scores, threshold=3)


# --- Print Results ---

print("\n--- LangGraph Evaluation Results ---")
print(f"📊 Mean Absolute Error (MAE): {final_mae:.4f}")
print("---")
print(f"🎯 Accuracy@1 (error <= 1.0): {acc_at_1:.2%}")
print(f"🎯 Accuracy@2 (error <= 2.0): {acc_at_2:.2%}")
print(f"🎯 Accuracy@3 (error <= 3.0): {acc_at_3:.2%}")
print("--------------------------------------")

Evaluating LangGraph in Batches:   0%|          | 0/17 [00:00<?, ?it/s]

[VALIDATION_INPUT] Validating input...
Pass id 0
Pass id 1
Pass id 2
Pass id 3
Pass id 4
An error occurred during graph execution: 0


Evaluating LangGraph in Batches:   6%|▌         | 1/17 [00:40<10:41, 40.11s/it]

[VALIDATION_INPUT] Validating input...
Pass id 0
Pass id 1
Pass id 2
Pass id 3
Pass id 4
An error occurred during graph execution: 0


Evaluating LangGraph in Batches:  12%|█▏        | 2/17 [01:21<10:09, 40.61s/it]

...

[VALIDATION_INPUT] Validating input...
Pass id 0
Pass id 1
Pass id 2
Pass id 3
Pass id 4
An error occurred during graph execution: 0


Evaluating LangGraph in Batches: 100%|██████████| 17/17 [11:48<00:00, 40.27s/it]

[VALIDATION_INPUT] Validating input...
Pass id 0
Pass id 1
An error occurred during graph execution: 0


Evaluating LangGraph in Batches: 18it [12:23, 41.29s/it]                        

--- Evaluation Complete ---

--- LangGraph Evaluation Results ---
📊 Mean Absolute Error (MAE): 6.3889
---
🎯 Accuracy@1 (error <= 1.0): 5.56%
🎯 Accuracy@2 (error <= 2.0): 5.56%
🎯 Accuracy@3 (error <= 3.0): 22.22%
--------------------------------------
